(acoustics-beampattern)=
# Beampattern

Although a spherical source is the simplest type of sonar source, the most common man-made sound sources used in underwater acoustics are planer transducers which have flat surfaces which oscillate to produce sound. This is in part due to the fact that a flat transducer is relatively easy to manufacture, but more importantly, the transducer transmits a majority of the sound in the direction perpendicular to the vibrating plane.  This is a useful property in many sonar systems in that this directionality can help determine the location of an object (fish, bubble, submarine, etc.)

(acoustics-SL_piston_far_field)=
## The circular piston transducer

Note that we said the transducer directs a "majority" of the sound in one direction and not "all" of the sound. In fact, the transducer sends different levels of sound in many directions and the directional dependence of the sound is refered to as the transducer's **beam pattern**. To help understand the concept of a beam pattern and it's properties, we will focus initially on one particular, common type of tranducer: the circular piston. This type of transducer is often a cylindrical piece of piezoelectric material to which an electrical signal is applied such that it expands and contracts along the axis of the cylinder. The causes sound to radiate from the circular face of the transducer. The structure of the sound field from this type of transducer can be fairly complicated and there is not a simple, mathematical expression for the direction and strength of the transmitted sound. There are, however, approximate mathematical descriptions that can accurately describe the transmitted sound under certain conditions. One of the most useful of these is to assume that the transducer can be approximated as a baffled circular piston. By "baffled," we mean that the face of the transducer is surrounded by an infinite, rigid surface. This simplifies the mathematics and allows us to calculate the amplitude and direction of the transmitted field.

In the following, the face of the transducer will be in the x-y plane and the z-axis is perpendicular to the face. The radius of the tranducer is _a_ and the field will be described in two ways; either a function of the distance, _r_, from the center of the face and the angle, $\theta$, between the direction of _r_ and the z-axis or as a function of distance from the plane of the transducer (range) and the perpendicular direction (cross-range.)

```{image} ../images/piston_diagram1.png
:width: 600px
```

 If the sound is measured far away from the face of the transducer, the intensity of the sound will be proportional to $1/r^2$ and the angular-dependence of the field can be expressed in a simple form that does not depend on the distance. When the reciever is sufficiently far away that these two properties are true, then the receiver is said to be in the **far-field** of the transducer. In the widget below, we've removed the $1/r^2$ dependence and normalized the intensity by the value measured on the z-axis or $\theta = 0^\circ$. As a result, the normalized intensity is 0 dB (or 1 on a linear scale.) The widget allows you to make the transducer larger or smaller by increasing it's radius and allows you to change the frequency of the sound being transmitted. You can also display the angular dependence of the transmitted sound on different plots and on both a linear and decibel scale.

In [1]:
# Import packages
import multiprocessing as mp
from functools import partial

import numpy as np
import matplotlib.pyplot as plt
from scipy.special import j1  # Bessel function of the first kind
import ipywidgets as widgets
from ipywidgets import interactive
import pandas as pd

In [2]:
# Constants
rho0 = 1
c = 1500
r = 10

# Create sliders for 'f' and 'a'
f_slider = widgets.IntSlider(
    min=1, max=200, step=1, value=30,
    description='Frequency (kHz)',
    style={'description_width': 'initial'},
)
a_slider = widgets.FloatSlider(
    min=0.01, max=1, step=0.005, value=0.1,readout=True,
    readout_format='.3f', description='Aperture Radius (m)',
    style={'description_width': 'initial'},
)

def far_field_intensity_cartesian_polar_calculation(f, a):
    # Calculate angular frequency
    omega = 2 * np.pi * f * 1000

    # Calculate wavenumber
    k = omega / c

    # Projection Intensity calculation
    theta = np.arange(-90, 90.1, 0.1)
    u = k * a * np.sin(theta * np.pi / 180)
    projInt = np.abs((rho0 * c * k * a * a / (2 * r)) * 
                     (2 * j1(u) / u)) ** 2

    # Handle division by zero at theta = 0
    ind = np.where(theta == 0)[0]
    axisInt = np.abs(rho0 * c * k * a * a / (2 * r)) ** 2
    projInt[ind] = axisInt

    return theta, projInt, axisInt

def update_far_field_intensity_cartesian_polar_plot(f, a, plotType, plotScale):
    # Calculate far field intensity
    theta, projInt, axisInt = far_field_intensity_cartesian_polar_calculation(f, a)

    # Close any existing plots
    plt.close('all') 

    # Plot in Cartesian
    if plotType == 'Cartesian':
        if plotScale == "Decibels":
            plt.plot(theta, 10 * np.log10(projInt / axisInt), linewidth=2)
            plt.ylim([-40, 0])
            plt.xlim([-90, 90])
            plt.xlabel('Angle (degrees)', fontsize=14)
            plt.ylabel('Beam Level (dB)', fontsize=14)
            plt.grid(True)
        else:
            plt.plot(theta, projInt / axisInt, linewidth=2)
            plt.ylim(10 ** (np.array([-40, 0]) / 10))
            plt.xlim([-90, 90])
            plt.xlabel('Angle (degrees)', fontsize=14)
            plt.ylabel('Beam Level', fontsize=14)
            plt.grid(True)

    # Plot in Polar
    elif plotType == 'Polar':
        if plotScale == "Decibels":
            plt.polar(theta * np.pi / 180, 10 * np.log10(projInt / axisInt), linewidth=2)
            ax = plt.gca()
            ax.set_ylim([-40, 0])
            ax.set_rticks([-40, -30, -20, -10, 0])
            ax.set_thetagrids([-90, -75, -60, -45, -30, -15, 0, 15, 30, 45, 60, 75, 90])
            ax.set_theta_zero_location('N')  # 'top'
            ax.set_thetalim([np.radians(-90), np.radians(90)])
            ax.set_title('Beam Pattern (dB)', fontsize=14)
        else:
            plt.polar(theta * np.pi / 180, projInt / axisInt, linewidth=2)
            ax = plt.gca()
            ax.set_ylim(10 ** (np.array([-40, 0]) / 10))
            ax.set_theta_zero_location('N')  # 'top'
            ax.set_thetalim([np.radians(-90), np.radians(90)])
            ax.set_title('Beam Pattern (Linear Scale)', fontsize=14)

    # Calculate beam widths
    ind3dB = np.where(10 * np.log10(projInt / axisInt) >= -3)[0]
    ind6dB = np.where(10 * np.log10(projInt / axisInt) >= -6)[0]
    ind10dB = np.where(10 * np.log10(projInt / axisInt) >= -10)[0]
    bw3dB = 2 * np.abs(theta[ind3dB[0]])
    bw6dB = 2 * np.abs(theta[ind6dB[0]])
    bw10dB = 2 * np.abs(theta[ind10dB[0]])

    # Define labels and values
    bwLabels = ['down 3 dB', 'down 6 dB', 'down 10 dB']
    bwValues = [bw3dB, bw6dB, bw10dB]

    # Create a DataFrame
    df = pd.DataFrame({
        'Beam Width (degrees)': bwValues,
        'Definition': bwLabels
    })

    # Print the DataFrame
    print(df)

# Create widgets for plot type and scale
plot_type_widget = widgets.Dropdown(
    options=['Cartesian', 'Polar'],
    value='Polar',
    description='Plot Type:'
)

plot_scale_widget = widgets.Dropdown(
    options=['Decibels', 'Linear'],
    value='Decibels',
    description='Scale:'
)

# Use interact to link widgets and plotting function
interactive_plot = interactive(
    update_far_field_intensity_cartesian_polar_plot,
    f=f_slider,
    a=a_slider,
    plotType=plot_type_widget,
    plotScale=plot_scale_widget,
)
display(interactive_plot)

interactive(children=(IntSlider(value=30, description='Frequency (kHz)', max=200, min=1, style=SliderStyle(des…

One of the things you'll notice as you decrease the frequency or decrease the radius, is that the central part of the beam pattern gets wider. This central portion of the beam pattern that includes the maximum at 0 degrees and is enclosed by the first null (points where the level gets very low) is called the **main lobe** or **main beam**. Most of the energy of the transmitted sound lies in this beam. As a result, most applications try to utilize just this beam. A key parameter for the transducer is width of this main beam or the **beam width**. The beam width is defined as the width of the beam when the field drops to a set level below the maximum value. This level is sometimes defined as 3 dB, 6 dB, or 10 dB which on a linear scale corresponds to 1/2, 1/4, or 1/10 of the maximum of the maximum value of the main lobe. For example, [SS510 transducer spec sheet](https://www.teledynemarine.com/en-us/products/SiteAssets/Odom/transducers/SMBB200-9.pdf) states that when operating at 200 kHz, the beam width is 9 degrees @ -3 dB. The table above shows the beamwidth defined relate to the different levels for the plotted beam pattern.

```{image} ../images/beam_diagram.png
:width: 400px
```

```{admonition} Exercise
For the SS510 transducer, the spec sheet says that at 200 kHz, the beam width is 9 degrees @ -3 dB. Using the widget, can you estimate the radius of the transducer?
``` 

The other beams or lobes are often referred to as **side lobes**. Notice that while the shape of the main lobe and the side lobes change significantly with frequency and transducer radius, the maximum level of the two sidelobes on either side of the main lobe is always at the same level, 17 dB, or 1/50 of the main lobe. This doesn't always hold true for all piston transducers but is a good approximation for the level of the first side lobe and is a useful rule of thumb for the far-field of the transducer. 

The field of the piston transducer is calculated below as a function of range from the transducer face and cross-range. While the calculation above shows how the field behaves as a function of angle relative to piston's center, this calculation shows how the field varies throughout the water in front of the sound source.

In [3]:
# Constants
rho0 = 1
c = 1500

# Create sliders for 'f' and 'a'
f_slider = widgets.IntSlider(
    min=1, max=50, step=1, value=30,
    description='Frequency (kHz)',
    style={'description_width': 'initial'},
)
a_slider = widgets.FloatSlider(
    min=0.1, max=1, step=0.05, value=0.1,
    description='Aperture Radius (m)',
    style={'description_width': 'initial'},
)

def far_field_intensity_x_y_plane_calculation(f, a, r, cr):
    # Calculate angular frequency
    omega = 2 * np.pi * f * 1000

    # Calculate wavenumber
    k = omega / c

    # Calculate sound pressure level in linear units
    SL_dB = 180
    SL_lin = 10**(SL_dB / 20)

    # Calculate the particle velocity amplitude
    U0 = SL_lin / (rho0 * c * k * a * a / 2)

    # Calculate meshgrids
    x = np.linspace(-cr, cr, 150)
    y = np.linspace(0, r, 100)
    X, Y = np.meshgrid(x, y)

    # Far field calculation
    thetaXY = np.arctan2(X, Y)
    rXY = np.sqrt(X**2 + Y**2)
    u1 = k * a * np.sin(thetaXY)
    u1[u1 == 0] = np.nan  # Avoid division by zero
    projIntXY = np.abs((rho0 * c * k * U0 * a * a / (2 * rXY)) *
                    (2 * j1(u1) / u1))**2

    return x, y, SL_dB, projIntXY

def update_far_field_intensity_x_y_plane_plot(f, a, plotScale):
    # Calculate far field intensity on x-y plane
    x, y, SL_dB, projIntXY = far_field_intensity_x_y_plane_calculation(f, a, r=10, cr=10)

    # Close any existing plots
    plt.close('all')

    # Plotting
    plt.figure(1)
    if plotScale == 'Decibels':
        plt.imshow(10 * np.log10(projIntXY), extent=(x.min(), x.max(), y.min(), y.max()), origin='lower')
        plt.colorbar(label='Intensity (dB re 1μPa)')
        plt.clim(SL_dB - 90, SL_dB)
    else:
        plt.imshow(projIntXY / 1e6, extent=(x.min(), x.max(), y.min(), y.max()), origin='lower')
        plt.colorbar(label='Intensity (μPa²)')
        plt.clim(10**((SL_dB - 90) / 10), SL_dB)
        
    plt.xlabel('Cross Range (m)')
    plt.ylabel('Range (m)')
    plt.gca().set_aspect('auto')
    plt.show()

# Use interact to link widgets and plotting function
interactive_plot = interactive(
    update_far_field_intensity_x_y_plane_plot,
    f=f_slider,
    a=a_slider,
    plotScale=plot_scale_widget,
)
display(interactive_plot)

interactive(children=(IntSlider(value=30, description='Frequency (kHz)', max=50, min=1, style=SliderStyle(desc…

# Other types of transducers

Another common transducer design is a transducer with a rectangular face. For the circular piston, we saw that as we increased the diameter of the piston face, the main beam became narrower in all directions. For the rectangular piston, as one of the axes of the rectangle is lengthened, the beam in the plane parallel to that axis will also become narrower while the beam perpendicular to that axis remains largely unchanged. This can be really useful in situations where you want a narrow beam in one direction and a wide beam in the other direction. Side scan sonars use a rectangular transducer to make it possible to image using a beam projected from the side of towed sonar that is wide in the vertical direction but very narrow in the direction of the towed. 